# Create Audio Dataset

In thhis notebook we are going to download, chop annd translate audios from a Youtube video.

Functions can be easily used for just a part of the process, for example, to chopand translate audios from a Wav file, or to ddownload audios  from YouTube video.

If runninng in Colab, pytube must be downloaded into your gdrive and installed from there, otherwise some bugs arose.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Move to where pytube is located
cd drive/MyDrive/pytube/

In [ ]:
!pip install -e .

In [ ]:
!pip install pydub
!pip install ffmpeg
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import os
import time
import json

import whisper

import pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence

from pytube import YouTube

In [ ]:
MODEL_SIZE = 'large'

In [ ]:
def download_from_youtube(
    url: str=URL,
    output_path: str='.',
):
  yt = YouTube(url)
  print(yt.title)
  # extract only audio
  audio = yt.streams.filter(only_audio=True).first()
  print(audio)
  # download the file
  out_file = audio.download(output_path=output_path)

  # save the file
  base, ext = os.path.splitext(out_file)
  new_file = base + '.mp3'
  os.rename(out_file, new_file)

  # result of success
  print(yt.title + " has been successfully downloaded.")

In [ ]:
download_from_youtube(URL)

In [ ]:
def load_to_wav(
    audio_path: str,
):
	if audio_path.endswith('.mp3'):
		audio = AudioSegment.from_file(
      audio_path,
      format='mp4'
    )
	elif audio_path.endswith('.wav'):
		audio = AudioSegment.from_file(
      audio_path,
      format='.wav',
    )

	audio = audio.set_channels(1)
	audio = audio.set_frame_rate(22050)
	audio.export(
      audio_path.split('.')[0] +'.wav',
      format = "wav"
  )

In [ ]:
load_to_wav('<PATH TO YOUR MP3 HERE>')

In [ ]:
def match_target_amplitude(
    chunk,
    target_dBFS: float=-22.0,
):
    """
    Takes an audio as an input and, given a  certain dB threshold will regain
    the audio.
    Returnns the regained audio.
    """
    change_in_dBFS = target_dBFS - chunk.dBFS

    return chunk.apply_gain(change_in_dBFS)


def split(
    audio_path: str,
    audios_directory: str,
    min_seconds: int=2,
    max_seconds: int=10,
    min_silence_length: int=200,
    silence_threshold: int= -40,
):
    """
    The function takes an audio, a destination(audios_directory) and min/max
    length, silence and silence threshold to chop it into smaller audios that
    match therequirements.

    It saves the chopped audios into the destination folder  with names  e.g.
    wav1.wav-wav100.wav
    """
    audio = AudioSegment.from_file(audio_path, format='wav')
    audio =  pydub.effects.normalize(audio)

    chunks = split_on_silence (
        audio,
        min_silence_len = min_silence_length,
        silence_thresh = silence_threshold,
    )

    for i, chunk in enumerate(chunks):
      # Descartamos archivos muy largos o muy cortos
      if min_seconds*1000 < len(chunk) < max_seconds*1000:
        chunk = match_target_amplitude(chunk)
        chunk.export(f"{audios_directory}/wav{str(i)}.wav", format = "wav")

In [ ]:
mkdir wavs

In [ ]:
split(
    audio_path="<PATH TO YOUR NNEWLY CREATED WAV FILE HERE>",
    audios_directory="wavs",
)

In [ ]:
cd wavs

In [ ]:
ls

wav11.wav  wav22.wav  wav29.wav  wav37.wav  wav46.wav  wav5.wav
wav12.wav  wav24.wav  wav33.wav  wav38.wav  wav49.wav  wav8.wav
wav18.wav  wav25.wav  wav35.wav  wav3.wav   wav52.wav
wav20.wav  wav28.wav  wav36.wav  wav41.wav  wav56.wav


In [ ]:
def dict_to_list(dict):
  lines_list=[]
  for key, value in dict.items():
    text_line=key + "|" + value
    lines_list.append(text_line)
  return lines_list

def whisper_speech_to_text(
    audios_directory: str='wavs',
    model_size: str=MODEL_SIZE,
):
  """
  Reads every wav in  audios_directory and creates a transcript.csv with  each
  audio filename and it's transcription.
  """
  model = whisper.load_model(model_size)

  filenames = os.listdir(audios_directory)
  total_files = len(filenames)
  audios_transcript={}

  for i, filename in enumerate(filenames):

    audio=os.path.join(audios_directory, filename)
    text = model.transcribe(audio)['text']

    if text is not None and len(text) > 0:
      audios_transcript[filename.split('.')[0]]=text
    print(f"Chunk {i} of {total_files} done.")
    print(f"Transcript: {text}")

  lines_list = dict_to_list(audios_transcript)

  audios_transcript_file="\n".join(lines_list)

  # (Para eso yo primero cree a mano un archivo vacio llamado "transcript.csv")
  with open('transcript.txt', 'w', encoding="utf-8") as file:
      file.write(audios_transcript_file)

In [ ]:
whisper_speech_to_text(audios_directory='wavs')